In [1]:
%reload_ext autoreload
%autoreload 2
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from mpmath import nsum, exp, inf
from main import get_all_retention_rates_for_cohort, get_data

# 4a

- $p_{c_k}$ : Active probability from constant retention rate. $p_{c_1} = 1$
- $r_c$ : Constant Retention rate

$$\mathbb{E}_c [user \space months] = \sum_{k=1}^{\inf}k \cdot p_{c_{k}} = \sum_{k=1}^{\inf}k \cdot r_c^{k-1}\cdot (1-r_c)$$

In [2]:
data = get_data()

constant_retention_rate = 0.56530017

active_probs_constant_retention = [(constant_retention_rate)**i for i in range(1000)]
#E[lifetime] = 1*retention_rate + 2*retention_rate^2 +..n*retention_rate^3
# print(active_probs_constant_retention)
# print(nsum(lambda n: n*active_probs_constant_retention[n-1], [1, 100]))
expected_value_constant_retention = sum([i*active_probs_constant_retention[i-1]*(1-constant_retention_rate) for i in range(1,len(active_probs_constant_retention))])
print(expected_value_constant_retention)

2.300437982687964


# 4b




*Should develop strategies to keep the customers in the second month*


*The expected lifetime of above 2 is heavily influenced by the loyal customers, that have been staying for many months and therefore driving the expected value up. On the other side we have seen big drops of customers after only one or two months, and we would therefore want to get more users past that critical point of two months. This could be done by more effectively showing the customer relevant content. It is however difficult to personalize content for a user that they have little data on, and one strategy could therefore be to gather data about the particular user from outside of Netflix itself to propose relevant, personalized content that may make the user stay past the critical point*

# 4c

In [3]:
def get_all_retention_rates_time_normalized(data):
    retention_rates_all_cohorts = []
    for i in range(12):
        all_retention_rates_for_cohort = (get_all_retention_rates_for_cohort(data,i))
        retention_rates_all_cohorts.append(all_retention_rates_for_cohort)
    for j in range(1,len(retention_rates_all_cohorts)):
        for i in range(j,len(retention_rates_all_cohorts[j])):
            retention_rates_all_cohorts[j][i-j] = retention_rates_all_cohorts[j][i]
            retention_rates_all_cohorts[j][i] = np.NaN
    return retention_rates_all_cohorts

retention_rates_cohort0 = np.array(get_all_retention_rates_for_cohort(data,0))

active_probs_changing_retention_cohort0 = retention_rates_cohort0
active_probs_changing_retention_cohort0[0] = 1
active_probs_changing_retention_cohort0 = active_probs_changing_retention_cohort0[~np.isnan(active_probs_changing_retention_cohort0)]
active_probs_changing_retention_cohort0 = np.cumprod(active_probs_changing_retention_cohort0)
print(active_probs_changing_retention_cohort0)

changing_retention_rates_probability_distribution = [(active_probs_changing_retention_cohort0[i-1])*(1-retention_rates_cohort0[i]) for i in range(1,len(active_probs_changing_retention_cohort0))]
expectation_value = np.arange(1,len(changing_retention_rates_probability_distribution)+1,1)@changing_retention_rates_probability_distribution
expectation_value


[1.00000000e+00 4.60858586e-01 3.40909091e-02 1.01010101e-02
 6.31313131e-04 0.00000000e+00]


1.5056818182033227

# 4d

The values for the different methods of calculating are radically different. Similarly as before we see here a sort of "abuse" of doing the average of averages in eliminating the relevance of the sample size. In this case in 4a we are using the average retention rate of the average retention rates of each month after customer acquisition.
To get the relevance of the newly acquired customers, also the ones that churned early, the last retention rate in 4c should be used as it is more realistic.
Using this, we realize that many churn after only one month, and the active probabilities fall quicker then in the first case

In [4]:
#4d
#Strategies for keeping customers during first month vs second month